CHANGELOG

v0.0 6/16/23:
- BasicAug
- batches_all

v0.1 6/19/23
- added in pearson correlation

v0.2 6/19/23:
- added hausdorff distance
- added IoU
- added AUC ROC
- added AUC PR

v0.3 6/24/23
- added access to jerry_losses.py

v0.4 7/3/2023
- added in restart run all

v0.5 7/4/2023
- added in if and elif clauses to check for prior existing files before running things

In [1]:
from jarvis.utils.general import gpus
gpus.autoselect()

import glob, numpy as np, pandas as pd, tensorflow as tf, os, scipy
from tensorflow.keras import Input, Model, layers
from pathlib import Path
from jarvis.train import models, custom
from jarvis.utils.display import imshow
from scipy.spatial.distance import directed_hausdorff
from IPython.display import clear_output, HTML, Javascript, display

import sys  
sys.path.append('/home/jjlou/Jerry/jerry_packages')
from jerry_utils import restart_kernel, load_dataset, load_dataset_v1
import jerry_losses, jerry_metrics 

[ 2024-01-09 12:57:11 ] CUDA_VISIBLE_DEVICES automatically set to: 2           


/home/jjlou/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
name = 'Attention_Unfrozen_EfficientNetV2L_data_v0.0.py'
root = '/home/jjlou/Jerry/wsi-arterio/arteriosclerotic_vessel_detection_and_fine_segmentation/Vessel_WallsLumen_Segmentation/data'

custom_objects = {
    'dice_all': jerry_metrics.dice_metric(cls=1),
    'hausdorff_all': jerry_metrics.hausdorff_metric(cls=1),
    'focal_dice_like_loss_multiclass_weighted': jerry_losses.focal_dice_like_loss_multiclass_weighted
}

In [3]:
Valid0 = ['D-436','D-870', 'D-343']
Valid1 = ['D-297', 'D-916', 'UCI-15-12']
Valid2 = ['D-322', 'D-562']

Valid = [Valid0, Valid1, Valid2]

In [4]:
def true_pred(true, pred, c):
    true = true.numpy()
    if c == None:
        true = true[..., 0]
        pred = np.argmax(pred, axis=-1)
    else:
        true = true[..., 0] == c
        pred = np.argmax(pred, axis=-1) == c
    return true, pred

In [5]:
calc_roc = tf.keras.metrics.AUC(curve='ROC')
calc_pr = tf.keras.metrics.AUC(curve='PR')
def AUC(y_true, y_pred, c=None):
    true, pred = true_pred(y_true, y_pred, c)
    calc_roc.update_state(true, pred)
    calc_pr.update_state(true, pred)
    roc = calc_roc.result().numpy()
    pr = calc_pr.result().numpy()
    calc_roc.reset_state()
    calc_pr.reset_state()
    return float(roc), float(pr)

In [6]:
m = tf.keras.metrics.MeanIoU(num_classes=3)
def mIoU(y_true, y_pred, c=None):
    true, pred = true_pred(y_true, y_pred, c)
    m.update_state(true, pred)
    r = m.result().numpy()
    m.reset_state()
    return float(r)

In [7]:
def hausdorff(y_true, y_pred, c=None):
    true, pred = true_pred(y_true, y_pred, c)
    true = np.squeeze(true)
    pred = np.squeeze(pred)
    d = directed_hausdorff(true, pred)[0]
    return d

In [8]:
def dice(y_true, y_pred, c=None):
    true, pred = true_pred(y_true, y_pred, c)
    true = true.astype('int64')
    A = np.count_nonzero(true & pred) * 2
    B = np.count_nonzero(true) + np.count_nonzero(pred)
    if np.count_nonzero(true) == 0 and np.count_nonzero(pred) == 0:
        A = 1
        B = 1
    return A / B

In [9]:
def pearson(y_true, y_pred, c=None):
    true, pred = true_pred(y_true, y_pred, c)
    true = true.flatten()
    pred = pred.flatten()
    pear, p = scipy.stats.pearsonr(pred,true)
    return pear, p

In [10]:
def vessel_lumen():
    # Vessel and lumen
    dsc_train_vl = []
    train_vl_pearson = []
    train_vl_p = []
    train_vl_hausdorff = []
    train_vl_mIoU = []
    train_vl_ROC = []
    train_vl_PR = []
    for x,y in train:
        logits = model.predict(x) 
        if type(logits) is dict:
            logits = logits['r1']
        dsc_train_vl.append(dice(y, logits)) # dice
        train_vl_hausdorff.append(hausdorff(y, logits)) # hausdorff
        train_vl_mIoU.append(mIoU(y, logits)) #mIoU
        roc, pr = AUC(y, logits) #AUC ROC and PR
        train_vl_ROC.append(roc)
        train_vl_PR.append(pr)
        pear, p = pearson(y, logits) # pearson
        train_vl_pearson.append(pear)
        train_vl_p.append(p) 

    dsc_valid_vl = []
    valid_vl_pearson = []
    valid_vl_p = []
    valid_vl_hausdorff = []
    valid_vl_mIoU = []
    valid_vl_ROC = []
    valid_vl_PR = []
    for x,y in valid:
        logits = model.predict(x) 
        if type(logits) is dict:
            logits = logits['r1']
        dsc_valid_vl.append(dice(y, logits))
        valid_vl_hausdorff.append(hausdorff(y, logits))
        valid_vl_mIoU.append(mIoU(y, logits))
        roc, pr = AUC(y, logits)
        valid_vl_ROC.append(roc)
        valid_vl_PR.append(pr)
        pear, p = pearson(y, logits)
        valid_vl_pearson.append(pear)
        valid_vl_p.append(p)            

    df_train_vl = pd.DataFrame(index=np.arange(len(dsc_train_vl)))
    df_valid_vl = pd.DataFrame(index=np.arange(len(dsc_valid_vl)))
    df_train_vl['Train dice score'] = dsc_train_vl
    df_valid_vl['Valid dice score'] = dsc_valid_vl
    df_train_vl['Train pearson'] = train_vl_pearson
    df_train_vl['Train p'] = train_vl_p
    df_valid_vl['Valid pearson'] = valid_vl_pearson
    df_valid_vl['Valid p'] = valid_vl_p
    df_train_vl['Train hausdorff'] = train_vl_hausdorff
    df_valid_vl['Valid hausdorff'] = valid_vl_hausdorff
    df_train_vl['Train mIoU'] = train_vl_mIoU
    df_valid_vl['Valid mIoU'] = valid_vl_mIoU
    df_train_vl['Train ROC'] = train_vl_ROC
    df_train_vl['Train PR'] = train_vl_PR
    df_valid_vl['Valid ROC'] = valid_vl_ROC
    df_valid_vl['Valid PR'] = valid_vl_PR
    df_train_vl.to_csv(f'{save_path}_train_vl.csv')
    df_valid_vl.to_csv(f'{save_path}_valid_vl.csv')

In [11]:
def vessel():
   # Vessel
    dsc_train_v = []
    train_v_pearson = []
    train_v_p = []
    train_v_hausdorff = []
    train_v_mIoU = []
    train_v_ROC = []
    train_v_PR = []
    for x,y in train:
        logits = model.predict(x) 
        if type(logits) is dict:
            logits = logits['r1']
        dsc_train_v.append(dice(y, logits,c=1))
        train_v_hausdorff.append(hausdorff(y, logits, c=1))
        train_v_mIoU.append(mIoU(y, logits, c=1))
        roc, pr = AUC(y, logits, c=1) 
        train_v_ROC.append(roc)
        train_v_PR.append(pr)
        pear, p = pearson(y, logits, c=1)
        train_v_pearson.append(pear)
        train_v_p.append(p) 

    dsc_valid_v = []
    valid_v_pearson = []
    valid_v_p = []
    valid_v_hausdorff = []
    valid_v_mIoU = []
    valid_v_ROC = []
    valid_v_PR = []
    for x,y in valid:
        logits = model.predict(x) 
        if type(logits) is dict:
            logits = logits['r1']
        dsc_valid_v.append(dice(y, logits,c=1))
        valid_v_hausdorff.append(hausdorff(y, logits, c=1))
        valid_v_mIoU.append(mIoU(y, logits, c=1))
        roc, pr = AUC(y, logits, c=1) 
        valid_v_ROC.append(roc)
        valid_v_PR.append(pr)
        pear, p = pearson(y, logits, c=1)
        valid_v_pearson.append(pear)
        valid_v_p.append(p) 

    df_train_v = pd.DataFrame(index=np.arange(len(dsc_train_v)))
    df_valid_v = pd.DataFrame(index=np.arange(len(dsc_valid_v)))
    df_train_v['Train dice score'] = dsc_train_v
    df_valid_v['Valid dice score'] = dsc_valid_v
    df_train_v['Train pearson'] = train_v_pearson
    df_train_v['Train p'] = train_v_p
    df_valid_v['Valid pearson'] = valid_v_pearson
    df_valid_v['Valid p'] = valid_v_p
    df_train_v['Train hausdorff'] = train_v_hausdorff
    df_valid_v['Valid hausdorff'] = valid_v_hausdorff
    df_train_v['Train mIoU'] = train_v_mIoU
    df_valid_v['Valid mIoU'] = valid_v_mIoU
    df_train_v['Train ROC'] = train_v_ROC
    df_train_v['Train PR'] = train_v_PR
    df_valid_v['Valid ROC'] = valid_v_ROC
    df_valid_v['Valid PR'] = valid_v_PR
    df_train_v.to_csv(f'{save_path}_train_v.csv')
    df_valid_v.to_csv(f'{save_path}_valid_v.csv') 

In [12]:
def lumen():
    # Lumen
    dsc_train_l = []
    train_l_pearson = []
    train_l_p = []
    train_l_hausdorff = []
    train_l_mIoU = []
    train_l_ROC = []
    train_l_PR = []
    for x,y in train:
        logits = model.predict(x) 
        if type(logits) is dict:
            logits = logits['r1']
        dsc_train_l.append(dice(y, logits,c=2))
        train_l_hausdorff.append(hausdorff(y, logits, c=2))
        train_l_mIoU.append(mIoU(y, logits, c=2))
        roc, pr = AUC(y, logits, c=2) 
        train_l_ROC.append(roc)
        train_l_PR.append(pr)
        pear, p = pearson(y, logits, c=2)
        train_l_pearson.append(pear)
        train_l_p.append(p) 

    dsc_valid_l = []
    valid_l_pearson = []
    valid_l_p = []
    valid_l_hausdorff = []
    valid_l_mIoU = []
    valid_l_ROC = []
    valid_l_PR = []
    for x,y in valid:
        logits = model.predict(x) 
        if type(logits) is dict:
            logits = logits['r1']
        dsc_valid_l.append(dice(y, logits,c=2))
        valid_l_hausdorff.append(hausdorff(y, logits, c=2))
        valid_l_mIoU.append(mIoU(y, logits, c=2))
        roc, pr = AUC(y, logits, c=2) 
        valid_l_ROC.append(roc)
        valid_l_PR.append(pr)
        pear, p = pearson(y, logits, c=2)
        valid_l_pearson.append(pear)
        valid_l_p.append(p) 

    df_train_l = pd.DataFrame(index=np.arange(len(dsc_train_l)))
    df_valid_l = pd.DataFrame(index=np.arange(len(dsc_valid_l)))
    df_train_l['Train dice score'] = dsc_train_l
    df_valid_l['Valid dice score'] = dsc_valid_l
    df_train_l['Train pearson'] = train_l_pearson
    df_train_l['Train p'] = train_l_p
    df_valid_l['Valid pearson'] = valid_l_pearson
    df_valid_l['Valid p'] = valid_l_p
    df_train_l['Train hausdorff'] = train_l_hausdorff
    df_valid_l['Valid hausdorff'] = valid_l_hausdorff
    df_train_l['Train mIoU'] = train_l_mIoU
    df_valid_l['Valid mIoU'] = valid_l_mIoU
    df_train_l['Train ROC'] = train_l_ROC
    df_train_l['Train PR'] = train_l_PR
    df_valid_l['Valid ROC'] = valid_l_ROC
    df_valid_l['Valid PR'] = valid_l_PR
    df_train_l.to_csv(f'{save_path}_train_l.csv')
    df_valid_l.to_csv(f'{save_path}_valid_l.csv') 

In [13]:
for v in Valid:
    v_path = f"{root}/K{Valid.index(v)}"
    train1 = glob.glob(f"{v_path}/Train_K{Valid.index(v)}_Color.Basic.Aug_x50_.43.17/*/*")
    train2 = glob.glob(f'{v_path}/Train_K{Valid.index(v)}_Gray.Basic.Aug_x10/*/*')
    train_path = train1 + train2
    valid_path = glob.glob(f"{v_path}/Valid_K{Valid.index(v)}_Color.BasicMorph.Aug/*") 
    model_path = f'{v_path}/models/{name}.hdf5'
    save_path = f'{v_path}/models/{name}_K{Valid.index(v)}'
    
    if os.path.exists(f'{save_path}_valid_l.csv'):
        continue
    else:
        # load data
        train = load_dataset_v1(train_path) 
        valid = load_dataset(valid_path)
        
        # load model
        model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)
    
        if not os.path.exists(f'{save_path}_valid_vl.csv'):
            vessel_lumen()
            vessel()
            lumen()
            restart_kernel()
            
        elif not os.path.exists(f'{save_path}_valid_v.csv'):
            vessel()
            lumen()
            restart_kernel()
        
        else:
            lumen()
            restart_kernel()

1/1 [==============================] - 0s 89ms/step


/home/jjlou/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


1/1 [==============================] - 0s 78ms/step


1/1 [==============================] - 0s 90ms/step


1/1 [==============================] - 0s 85ms/step


...

1/1 [==============================] - 0s 87ms/step


1/1 [==============================] - 0s 88ms/step


1/1 [==============================] - 0s 90ms/step


1/1 [==============================] - 0s 126ms/step


OutOfRangeError: {{function_node __wrapped____EagerConst_device_/job:localhost/replica:0/task:0/device:GPU:0}} End of sequence